In [1]:
# Gerekli kütüphaneleri yükle
!pip install -qU chromadb sentence-transformers langchain langchain_community langchain-text-splitters accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 31.2 MB

In [2]:
import os
import time
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch # GPU kontrolü için

print("Kütüphaneler yüklendi.")

# --- 1. Ayarlar ---
# Veri dosyalarının bulunduğu Kaggle yolu (kendi yüklediğiniz klasör adıyla değiştirin)
# Örnek: Eğer dosyaları "resmi-gazete-txt" adlı bir datasete yüklediyseniz:
# data_folder = "/kaggle/input/resmi-gazete-txt"
# Eğer direkt upload ettiyseniz ve Kaggle bir isim verdiyse o ismi kullanın.
# Sağdaki Input panelinden klasör ismini kontrol edebilirsiniz.
DATA_FOLDER = "/kaggle/input/gazete-txt" # <-- BURAYI GÜNCELLEYİN!

# ChromaDB'nin kaydedileceği yer (Kaggle çalışma dizini)
CHROMA_PERSIST_DIR = "/kaggle/working/chroma_db_bge_m3"
# ChromaDB Koleksiyon adı
CHROMA_COLLECTION_NAME = "resmi_gazete_bge_m3"

# Embedding Modeli
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"

# Metin Bölme Ayarları
CHUNK_SIZE = 1000 # Her bir parçanın karakter sayısı (deneyerek optimize edilebilir)
CHUNK_OVERLAP = 150 # Parçalar arası karakter örtüşmesi

# Cihazı Ayarla (GPU varsa CUDA kullan, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")
if not torch.cuda.is_available():
    print("UYARI: GPU bulunamadı, embedding işlemi CPU üzerinde yavaş olabilir.")

# --- 2. Veri Yükleme ve Hazırlama ---
print(f"\nVeri klasörü kontrol ediliyor: {DATA_FOLDER}")
if not os.path.exists(DATA_FOLDER):
    raise FileNotFoundError(f"HATA: Belirtilen veri klasörü bulunamadı: {DATA_FOLDER}. "
                          "Lütfen DATA_FOLDER değişkenini doğru ayarladığınızdan emin olun.")

all_docs = []
doc_count = 0
txt_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".txt")]

if not txt_files:
     raise FileNotFoundError(f"HATA: {DATA_FOLDER} içinde '.txt' uzantılı dosya bulunamadı.")

print(f"{len(txt_files)} adet .txt dosyası bulundu. Okunuyor...")

for filename in txt_files:
    file_path = os.path.join(DATA_FOLDER, filename)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Langchain Document objesi oluşturuyoruz (metadata ile)
            # Langchain'in kendi Document yapısını kullanmak yerine string listesi de olurdu
            # ama metadata eklemek için bu yapı kullanışlı.
            # Şimdilik basit metadata ekleyelim:
            all_docs.append({"page_content": content, "metadata": {"source": filename}})
            doc_count += 1
            print(f" - Okundu: {filename}")
    except Exception as e:
        print(f"HATA: {filename} okunurken hata oluştu: {e}")

print(f"\nToplam {doc_count} doküman başarıyla okundu.")

# --- 3. Metin Bölme (Chunking) ---
print("\nMetinler parçalara (chunk) ayrılıyor...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    add_start_index=True, # Her parçanın başlangıç indeksini metadata'ya ekler
)

chunks = []
for doc in all_docs:
    # text_splitter.create_documents direkt Document objeleriyle çalışır
    # Bizim yapımız biraz farklı olduğu için split_text kullanalım
    doc_content = doc["page_content"]
    doc_metadata = doc["metadata"]
    
    split_texts = text_splitter.split_text(doc_content)
    
    for i, text_chunk in enumerate(split_texts):
        # Her chunk için metadata'yı genişletelim
        chunk_metadata = doc_metadata.copy()
        chunk_metadata["chunk_index"] = i 
        # İleride chunk'ın hangi dokümanın hangi parçası olduğunu bilmek önemli
        
        chunks.append({"page_content": text_chunk, "metadata": chunk_metadata})

print(f"Toplam {len(chunks)} adet metin parçası (chunk) oluşturuldu.")

# Bellek optimizasyonu için orijinal dokümanları silebiliriz
del all_docs

# --- 4. Embedding Modelini Yükleme ---
print(f"\nEmbedding modeli yükleniyor: {EMBEDDING_MODEL_NAME}...")
# Modeli belirtilen cihaza (GPU/CPU) yükle
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=device)
print("Embedding modeli yüklendi.")

# --- 5. Embedding Oluşturma ---
print("\nMetin parçaları için embeddingler oluşturuluyor (Bu işlem zaman alabilir)...")

# Sadece metin içeriklerini alıp listeye çevir
texts_to_embed = [chunk["page_content"] for chunk in chunks]

start_time = time.time()
# Modeli kullanarak embeddingleri hesapla
# show_progress_bar=True ile ilerlemeyi gör
embeddings = embedding_model.encode(
    texts_to_embed,
    show_progress_bar=True,
    batch_size=32 # GPU belleğine göre ayarlanabilir
)
end_time = time.time()

print(f"Embedding oluşturma tamamlandı. Süre: {end_time - start_time:.2f} saniye.")
print(f"Oluşturulan embedding matris boyutu: {embeddings.shape}") # (chunk_sayisi, embedding_boyutu)

# Bellek Optimizasyonu
del texts_to_embed

# --- 6. ChromaDB Kurulumu ve Veri Yükleme ---
print(f"\nChromaDB kuruluyor ve veriler yükleniyor...")
print(f"Veritabanı yolu: {CHROMA_PERSIST_DIR}")

# PersistentClient ile veritabanını diske kaydet
# Eğer klasör varsa içeriğiyle birlikte yükler, yoksa oluşturur.
chroma_client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIR)

# Koleksiyonu al veya oluştur (Embedding fonksiyonu belirtmiyoruz, çünkü embeddingleri biz sağladık)
# ÖNEMLİ: Eğer daha önce aynı isimde farklı bir embedding modeliyle koleksiyon oluşturduysanız
# sorun yaşamamak için ya eski klasörü silin ya da farklı bir koleksiyon adı kullanın.
collection = chroma_client.get_or_create_collection(
    name=CHROMA_COLLECTION_NAME,
    # metadata={"hnsw:space": "cosine"} # BGE modelleri genellikle cosine similarity ile iyi çalışır, Chroma varsayılanı L2'dir. İsteğe bağlı.
)
print(f"'{CHROMA_COLLECTION_NAME}' koleksiyonu alındı veya oluşturuldu.")

# ChromaDB'ye eklemek için ID'ler ve metadatalar oluşturalım
ids = [f"{chunk['metadata']['source']}_chunk_{chunk['metadata']['chunk_index']}" for chunk in chunks]
metadatas = [chunk["metadata"] for chunk in chunks]
documents = [chunk["page_content"] for chunk in chunks] # ChromaDB dökümanları da saklar

# Veriyi ChromaDB'ye ekle
# Eğer çok fazla chunk varsa (örn. > 50k), batch'ler halinde eklemek daha iyi olabilir.
# Şu anki miktar için tek seferde eklemek sorun olmamalı.
print(f"{len(ids)} adet chunk ChromaDB'ye ekleniyor...")
try:
    collection.add(
        embeddings=embeddings.tolist(), # NumPy array'i listeye çevir
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    print("Veriler başarıyla ChromaDB'ye eklendi.")
except Exception as e:
    print(f"HATA: ChromaDB'ye veri eklenirken sorun oluştu: {e}")
    # Olası sorun: Aynı ID ile tekrar ekleme yapmaya çalışmak.
    # Eğer kodu tekrar çalıştırıyorsanız ve hata alıyorsanız, önce /kaggle/working/chroma_db_bge_m3 klasörünü silmeyi deneyin.

# Eklenen öğe sayısını kontrol et
count = collection.count()
print(f"Koleksiyondaki toplam öğe sayısı: {count}")

# Bellek Optimizasyonu
del embeddings, documents, metadatas, ids, chunks

# --- 7. Sonuç ve İndirme ---
print(f"\nİşlem tamamlandı!")
print(f"ChromaDB vektör deposu '{CHROMA_PERSIST_DIR}' klasörüne kaydedildi.")
print("\nSonraki Adımlar:")
print("1. Bu Kaggle Notebook'unu 'Save Version' ile kaydedin (Type: 'Run All', Save Output: 'Yes').")
print("2. Kaydetme işlemi bittikten sonra, Notebook versiyon sayfasının 'Output' sekmesine gidin.")
print(f"3. Orada '{os.path.basename(CHROMA_PERSIST_DIR)}' adlı klasörü bulun ve indirin.")
print("4. İndirdiğiniz bu klasörü lokal makinenizde RAG projenizde kullanabilirsiniz.")
print("\nLokalde Kullanım Örneği:")
print("```python")
print("import chromadb")
print("# İndirdiğiniz klasörün yolunu verin")
print(f"LOCAL_CHROMA_PATH = '/path/to/your/downloaded/{os.path.basename(CHROMA_PERSIST_DIR)}'")
print(f"PERSISTENT_CLIENT = chromadb.PersistentClient(path=LOCAL_CHROMA_PATH)")
print(f"COLLECTION = PERSISTENT_CLIENT.get_collection(name='{CHROMA_COLLECTION_NAME}')")
print("# Sorgu yapmak için yine BAAI/bge-m3 modelini kullanmalısınız!")
print("# query_embedding = embedding_model.encode(['Sorgunuz buraya'])")
print("# results = COLLECTION.query(query_embeddings=query_embedding, n_results=5)")
print("# print(results)")
print("```")

2025-04-23 11:00:28.258729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745406028.441747      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745406028.497923      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Kütüphaneler yüklendi.
Kullanılan Cihaz: cuda

Veri klasörü kontrol ediliyor: /kaggle/input/gazete-txt
5 adet .txt dosyası bulundu. Okunuyor...
 - Okundu: 20250410.txt
 - Okundu: 20250411.txt
 - Okundu: 20250407.txt
 - Okundu: 20250408.txt
 - Okundu: 20250409.txt

Toplam 5 doküman başarıyla okundu.

Metinler parçalara (chunk) ayrılıyor...
Toplam 388 adet metin parçası (chunk) oluşturuldu.

Embedding modeli yükleniyor: BAAI/bge-m3...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedding modeli yüklendi.

Metin parçaları için embeddingler oluşturuluyor (Bu işlem zaman alabilir)...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding oluşturma tamamlandı. Süre: 15.04 saniye.
Oluşturulan embedding matris boyutu: (388, 1024)

ChromaDB kuruluyor ve veriler yükleniyor...
Veritabanı yolu: /kaggle/working/chroma_db_bge_m3
'resmi_gazete_bge_m3' koleksiyonu alındı veya oluşturuldu.
388 adet chunk ChromaDB'ye ekleniyor...
Veriler başarıyla ChromaDB'ye eklendi.
Koleksiyondaki toplam öğe sayısı: 388

İşlem tamamlandı!
ChromaDB vektör deposu '/kaggle/working/chroma_db_bge_m3' klasörüne kaydedildi.

Sonraki Adımlar:
1. Bu Kaggle Notebook'unu 'Save Version' ile kaydedin (Type: 'Run All', Save Output: 'Yes').
2. Kaydetme işlemi bittikten sonra, Notebook versiyon sayfasının 'Output' sekmesine gidin.
3. Orada 'chroma_db_bge_m3' adlı klasörü bulun ve indirin.
4. İndirdiğiniz bu klasörü lokal makinenizde RAG projenizde kullanabilirsiniz.

Lokalde Kullanım Örneği:
```python
import chromadb
# İndirdiğiniz klasörün yolunu verin
LOCAL_CHROMA_PATH = '/path/to/your/downloaded/chroma_db_bge_m3'
PERSISTENT_CLIENT = chromadb.Persist

In [3]:
# --- 7. ChromaDB Kaydını Doğrulama ve Zipleme ---
print(f"\nİşlem tamamlandı!")
print(f"ChromaDB vektör deposu '{CHROMA_PERSIST_DIR}' klasörüne kaydedildi.")

# Eklenen öğe sayısını tekrar kontrol edelim (isteğe bağlı ama iyi bir kontrol)
try:
    final_count = collection.count()
    print(f"Koleksiyondaki öğe sayısı doğrulandı: {final_count}")
    if final_count == 0 and len(chunks) > 0: # Eğer chunk varsa ama koleksiyon boşsa uyarı ver
         print("UYARI: Koleksiyonda hiç öğe bulunamadı. Ekleme sırasında bir sorun olmuş olabilir.")
except Exception as e:
    print(f"UYARI: Koleksiyon sayımı kontrol edilirken hata: {e}")


# --- 8. Output Klasörünü Zipleme ---
import zipfile
import shutil # shutil.make_archive daha kolay olabilir

ZIP_FILE_NAME = f"{os.path.basename(CHROMA_PERSIST_DIR)}.zip"
ZIP_FILE_PATH = os.path.join("/kaggle/working/", ZIP_FILE_NAME) # Zip dosyasını /kaggle/working içine kaydet

print(f"\n'{CHROMA_PERSIST_DIR}' klasörü zipleniyor...")
print(f"Kaynak: {CHROMA_PERSIST_DIR}")
print(f"Hedef Zip: {ZIP_FILE_PATH}")

try:
    # shutil.make_archive kullanımı daha basit
    shutil.make_archive(
        base_name=os.path.join("/kaggle/working/", os.path.basename(CHROMA_PERSIST_DIR)), # Zip dosyasının tam yolu (.zip olmadan)
        format='zip',        # Arşiv formatı
        root_dir=os.path.dirname(CHROMA_PERSIST_DIR), # Arşivlenecek klasörün üst dizini (/kaggle/working)
        base_dir=os.path.basename(CHROMA_PERSIST_DIR) # Arşivlenecek klasörün adı (chroma_db_bge_m3)
    )

    # # Alternatif: zipfile modülü ile manuel zipleme (daha detaylı kontrol isterseniz)
    # with zipfile.ZipFile(ZIP_FILE_PATH, 'w', zipfile.ZIP_DEFLATED) as zipf:
    #     # base_dir = os.path.dirname(CHROMA_PERSIST_DIR) # /kaggle/working
    #     for root, dirs, files in os.walk(CHROMA_PERSIST_DIR):
    #         for file in files:
    #             file_path = os.path.join(root, file)
    #             # Zip içindeki yolu ayarlamak için:
    #             # /kaggle/working/chroma_db_bge_m3/file.txt -> chroma_db_bge_m3/file.txt
    #             archive_name = os.path.relpath(file_path, os.path.dirname(CHROMA_PERSIST_DIR))
    #             zipf.write(file_path, arcname=archive_name)

    print(f"'{ZIP_FILE_PATH}' başarıyla oluşturuldu.")

    # İsteğe bağlı: Zipleme sonrası orijinal klasörü silerek disk alanı açabilirsiniz
    # Ancak output'ta hem klasör hem zip olsun isterseniz bu satırı yorumlayın/silin
    # print(f"Orijinal klasör siliniyor: {CHROMA_PERSIST_DIR}")
    # shutil.rmtree(CHROMA_PERSIST_DIR)

except Exception as e:
    print(f"HATA: Zipleme sırasında bir hata oluştu: {e}")


# --- 9. Sonuç ve İndirme Talimatları ---
print("\nSonraki Adımlar:")
print("1. Bu Kaggle Notebook'unu 'Save Version' ile kaydedin (Type: 'Run All', Save Output: 'Yes').")
print("2. Kaydetme işlemi bittikten sonra, Notebook versiyon sayfasının 'Output' sekmesine gidin.")
print(f"3. Orada '{ZIP_FILE_NAME}' adlı zip dosyasını bulun ve indirin.")
print("4. İndirdiğiniz zip dosyasını lokal makinenizde uygun bir yere açın.")
print("5. Açtığınız klasör ('chroma_db_bge_m3' adında olmalı), lokal RAG projenizde kullanacağınız vektör deposudur.")
print("\nLokalde Kullanım Örneği:")
print("```python")
print("import chromadb")
print("# Açtığınız klasörün yolunu verin")
print(f"LOCAL_CHROMA_PATH = '/path/to/your/extracted/{os.path.basename(CHROMA_PERSIST_DIR)}'") # Zip'ten çıkan klasörün yolu
print(f"PERSISTENT_CLIENT = chromadb.PersistentClient(path=LOCAL_CHROMA_PATH)")
print(f"COLLECTION = PERSISTENT_CLIENT.get_collection(name='{CHROMA_COLLECTION_NAME}')")
print("# Sorgu yapmak için yine BAAI/bge-m3 modelini kullanmalısınız!")
print("# from sentence_transformers import SentenceTransformer")
print("# embedding_model = SentenceTransformer('BAAI/bge-m3') # Modeli yükle")
print("# query_embedding = embedding_model.encode(['Sorgunuz buraya']).tolist()")
print("# results = COLLECTION.query(query_embeddings=query_embedding, n_results=5)")
print("# print(results)")
print("```")


İşlem tamamlandı!
ChromaDB vektör deposu '/kaggle/working/chroma_db_bge_m3' klasörüne kaydedildi.
Koleksiyondaki öğe sayısı doğrulandı: 388

'/kaggle/working/chroma_db_bge_m3' klasörü zipleniyor...
Kaynak: /kaggle/working/chroma_db_bge_m3
Hedef Zip: /kaggle/working/chroma_db_bge_m3.zip
'/kaggle/working/chroma_db_bge_m3.zip' başarıyla oluşturuldu.

Sonraki Adımlar:
1. Bu Kaggle Notebook'unu 'Save Version' ile kaydedin (Type: 'Run All', Save Output: 'Yes').
2. Kaydetme işlemi bittikten sonra, Notebook versiyon sayfasının 'Output' sekmesine gidin.
3. Orada 'chroma_db_bge_m3.zip' adlı zip dosyasını bulun ve indirin.
4. İndirdiğiniz zip dosyasını lokal makinenizde uygun bir yere açın.
5. Açtığınız klasör ('chroma_db_bge_m3' adında olmalı), lokal RAG projenizde kullanacağınız vektör deposudur.

Lokalde Kullanım Örneği:
```python
import chromadb
# Açtığınız klasörün yolunu verin
LOCAL_CHROMA_PATH = '/path/to/your/extracted/chroma_db_bge_m3'
PERSISTENT_CLIENT = chromadb.PersistentClient(path=